In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11802,2024-10-23,Japão B2.League,07:05,Kobe,Shizuoka,1.42,2.57,157.5,1.80,1.86,-6.5,1.94,1.74,CdqpLuYA,0.704225,0.389105,0.555556,0.537634,0.093330,127.616,27.737438,0.217351,2.4,1.341641,0.559017,110.00,1.540,0.570658,0.370557,28.0,251.628,150.989483,0.600050,1.2,1.643168,1.369306,144.57,3.648,2.521868,0.691302,-16.0,88,79,1.25,1.83,122.420,246.738,0.407605,0.023184,0.076859,0.51,0.102,4.117647,0.718571,0.7,-0.018571,0.05,0.010,157.000000,0.395080,0.5,0.104920
11803,2024-10-23,Japão B2.League,07:05,Toyama,Fukushima,1.24,3.52,162.5,1.86,1.80,-9.5,1.92,1.75,GUkyN13b,0.806452,0.284091,0.537634,0.555556,0.090543,191.570,92.099657,0.480762,1.2,1.643168,1.369306,99.84,2.634,1.698140,0.644700,4.0,167.580,42.580477,0.254090,1.2,1.643168,1.369306,171.10,2.260,0.685930,0.303509,-9.0,78,58,1.28,2.95,376.112,228.768,0.677396,0.023184,0.065509,-5.00,-1.000,-0.240000,0.272816,0.0,-0.272816,-2.51,-0.502,-5.019920,0.449213,0.4,-0.049213
11804,2024-10-23,Japão B2.League,07:35,Shinshu,Fukuoka,1.38,2.72,152.5,1.84,1.82,-6.5,1.88,1.78,2RNODcQp,0.724638,0.367647,0.543478,0.549451,0.092285,152.248,72.005607,0.472949,2.4,1.341641,0.559017,107.69,2.000,1.468094,0.734047,41.0,152.122,57.336033,0.376908,1.2,1.643168,1.369306,219.78,1.978,0.806951,0.407963,9.0,89,74,1.21,2.97,135.826,130.602,0.462206,0.007728,0.038640,-1.07,-0.214,-1.775701,0.486817,0.3,-0.186817,-0.59,-0.118,-14.576271,0.681871,0.7,0.018129
11805,2024-10-23,Lituânia King Mindaugas Cup,12:30,Juventus,Jonava,1.30,3.07,173.5,1.83,1.83,-8.5,1.95,1.72,OWYaH1U6,0.769231,0.325733,0.546448,0.546448,0.094964,112.634,6.634326,0.058902,1.8,1.643168,0.912871,113.10,1.276,0.087063,0.068231,28.0,442.114,300.222103,0.679060,1.2,1.643168,1.369306,212.08,5.474,4.010914,0.732721,-72.0,87,88,1.30,2.41,106.754,333.356,0.572805,0.000000,0.088629,-0.30,-0.060,-5.000000,0.000000,0.0,0.000000,-3.23,-0.646,-3.204334,0.000000,0.0,0.000000
11806,2024-10-23,Polônia 1. Liga,13:00,Krosno,Bydgoszcz,1.45,2.48,155.5,1.85,1.81,-5.5,1.90,1.76,dr0uPQb4,0.689655,0.403226,0.540541,0.552486,0.092881,132.886,32.713743,0.246179,2.4,1.341641,0.559017,132.24,1.696,0.324392,0.191269,39.0,146.834,40.267879,0.274241,1.2,1.643168,1.369306,171.08,1.922,0.647163,0.336713,-7.0,87,91,1.52,1.88,115.680,152.404,0.370646,0.015456,0.054096,0.60,0.120,3.750000,0.753563,0.9,0.146437,-0.29,-0.058,-25.517241,0.583320,0.4,-0.183320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11908,2024-10-23,Europa Copa Europeia Da Fiba,14:00,Tofas,Braunschweig,1.22,4.11,167.5,1.87,1.79,-10.5,2.02,1.71,v50imdsI,0.819672,0.243309,0.534759,0.558659,0.062981,198.960,100.882536,0.507049,1.8,1.643168,0.912871,101.76,2.616,1.597288,0.610584,25.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,96,94,1.06,1.23,159.158,0.000,0.766806,0.030912,0.117535,1.52,0.304,0.723684,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11909,2024-10-23,Europa Copa Europeia Da Fiba,14:00,Levice,Bursaspor,2.51,1.50,168.5,1.86,1.80,3.5,2.00,1.72,WE7aLtLi,0.398406,0.666667,0.537634,0.555556,0.065073,243.264,145.595320,0.598507,1.2,1.64316

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11818,07:05,Japão B.League,Nagasaki,Osaka,1.59,Back Home
11821,07:05,Japão B.League,Diamond Dolphins,NeoPhoenix,2.15,Back Home
11828,21:00,Argentina Liga A,Zarate,Peñarol,1.77,Back Home
11856,07:00,Coreia Do Sul Kbl,KCC Egis,Goyang Sono,1.68,Back Home
11873,22:00,Eua Nba,Utah Jazz,Memphis Grizzlies,2.27,Back Home
11880,20:00,Eua Nba,Detroit Pistons,Indiana Pacers,2.56,Back Home
